# **Statistical Analysis of Extreme Events**
# Peaks Over Threshold (POT) - Poisson & Generalized Pareto distributions

In [ ]:
import os
import os.path as op
import sys 

# arrays
import numpy as np
import pandas as pd
import xarray as xr
import datetime
from numpy.random import multivariate_normal
from scipy.stats import poisson, genpareto, genextreme
from scipy.optimize import curve_fit
from scipy.io import loadmat

# plots
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# append ddviewer to path
sys.path.insert(0, op.join(os.getcwd(), '..','..'))

from lib.extremes import *
from lib.config import *

In [ ]:
from IPython.display import Image
Image(url='../../lib/resources/POT.png', width=700)

## **Load input database** <a class="anchor" id="1"></a>

In [ ]:
# Load database (.csv)
path = op.join(os.getcwd(), '..','storage')
file = 'TWL_1139.csv'

In [ ]:
data = pd.read_csv(
    op.join(path, file), parse_dates=['time']
)
data = data.set_index('time').dropna()

## **Data visualization** <a class="anchor" id="2"></a>

In [ ]:
var = 'TWL'
data = data[[var]]

data['year'] = data.index.year
data['month'] = data.index.month
data['day'] = data.index.day

data = data.resample('1H').ffill()

### Box-plot of monthly data <a class="anchor" id="21"></a>

In [ ]:
# Use plotly library for plotting boxplot
fig = px.box(data, x="month", y=var, notched=True)
fig.show()

### Monthly maxima <a class="anchor" id="22"></a>

In [ ]:
df_mm = data.groupby(by=['year','month'])[var].max().dropna().reset_index()
df_mm = pd.merge(df_mm, data, how='inner', on=['year', 'month', var])
df_mm['date'] = pd.to_datetime(df_mm[['year', 'month', 'day']], errors='coerce')
df_mm = df_mm.set_index('date')

In [ ]:
fig = px.line(df_mm[var], width=1200, height=400)
fig.show()

## **Fit Historical Peaks over Threshold to Pareto-Poisson distribution** <a class="anchor" id="3"></a>

### Eliminate years with incomplete months

In [ ]:
data['day_id'] = 1
data_few = data.groupby('year').sum().reset_index()
data_few = data_few.loc[data_few['day_id'] > 300]

In [ ]:
data = data[data['year'].isin(data_few['year'].values)]

### Calculate POT

In [ ]:
# Set threshold 
th = np.percentile(data[var], 98) 
ie = 5   # days between independent peaks

# Calculate data over threshold
data_pot = data.loc[data[var] >= th]

# Days between consecutive events
ix_space = np.diff(data_pot.index).astype('timedelta64[h]').astype(int)
ix_space = list(ix_space) + [ie] # include last event

# Get independent peaks
idx = []
for nt in np.where(ix_space)[0]:
    day = data_pot.index[nt]
    rangeday = np.arange(day-datetime.timedelta(days=ie), day+datetime.timedelta(days=ie), datetime.timedelta(hours=1))
    if data_pot[var].iloc[nt] == np.max(data[var].loc[rangeday]):
        idx.append(nt)

data_pot = data_pot.iloc[idx]


In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = data.index, y = data[var],
        mode ='lines', name = 'Historical {0} (m)'.format(var),
    )
)
fig.add_trace(
    go.Scatter(
        x = data_pot.index, y = data_pot[var],
        mode ='markers', name = 'Peaks Over Threshold',
    )
)
fig.add_shape(
    type="line",
    x0=0, x1=1, xref='paper',
    y0=th,  y1=th, yref='y',
    line=dict(
        color="gray",
        dash="dash",
    )
)
fig.update_layout(    
    xaxis_title = "time",
    yaxis_title = "{0} (m)".format(var),
    yaxis=dict(rangemode='nonnegative')
)
fig.show()


### Fit POT Frequency to Poisson distribution <a class="anchor" id="32"></a>

In [ ]:
# get number of events each year 
y_events = data_pot.groupby(by=[data_pot.index.year]).count()[var].values

# get frequency
N, freq = np.unique(y_events, return_counts=True)

# optimize poisson lambda parameter
params = np.mean(y_events)
print('lambda Poisson : {0}'.format(np.float(params)))

# freeze poisson distribution
rv_poi = poisson(params)

# generate some values from poisson to plot
x = np.arange(rv_poi.ppf(0), rv_poi.ppf(0.999))
y = rv_poi.pmf(x) 

In [ ]:
# Plot POISSON PMF vs data frequency histogram
print(freq)

fig = go.Figure()
fig.add_trace(go.Histogram(x=y_events, histnorm='probability density', name='Historical Hs'))
fig.add_trace(go.Bar(x=x, y=y, name='PMF'))

fig.update_layout(    
    xaxis_title = "x",
    yaxis_title = "P(x)",
    title = 'Poisson Probability Mass Function',
    yaxis=dict(rangemode='nonnegative')
)
fig.show()


### Simulate extreme values from Poisson distribution <a class="anchor" id="34"></a>

In [ ]:
# simulate some values for this Poisson
size_sim = 1000
var_sim = rv_poi.rvs(size=size_sim)

# Plot POI pdf vs simulated data
fig = go.Figure()
fig.add_trace(go.Histogram(x=var_sim, histnorm='probability density', name='Simulation Hs'))
fig.add_trace(go.Bar(x=x, y=y, name='PMF'))

fig.update_layout(    
    xaxis_title = "x",
    yaxis_title = "P(x)",
    title = 'Poisson Probability Mass Function',
    yaxis=dict(rangemode='nonnegative')
)
fig.show()


### Fit POT Intensity to Generalized Pareto distribution <a class="anchor" id="32"></a>

In [ ]:
data_pot

In [ ]:
# get event max intensity (for hydrologic year)
imax = data_pot.groupby(by=[data_pot.index.year]).max()
intensity = data_pot[var].values - th

# fit data to GPD
shape, loc, scale = genpareto.fit(intensity, floc=0)
print(shape, loc, scale)

# negative loglikelihood
nLogL = genpareto.nnlf((shape, loc, scale), intensity)

# GPD parameters
theta = (shape, loc, scale)

# freeze GPD with parameters, get GPD PDF
rv_gpd = genpareto(shape, loc, scale)  
x = np.linspace(rv_gpd.ppf(0), rv_gpd.ppf(1), 1000)
y = rv_gpd.pdf(x)


In [ ]:
# Plot GPD PDF vs data probability density histogram

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='PDF', marker_color='black'))
fig.add_trace(go.Histogram(x=intensity.reshape(-1), nbinsx=10,histnorm='probability density', name='Historical'))

fig.update_layout(    
    xaxis_title = "x",
    yaxis_title = "P(x)",
    title = 'GPD Probability Density Function',
    yaxis=dict(rangemode='nonnegative')
)
fig.update_xaxes(range=[0, x.max()])
fig.show()

### Simulate extreme values from Pareto distribution <a class="anchor" id="34"></a>

In [ ]:
# simulate some values for this GPD
var_sim = rv_gpd.rvs(size=size_sim)

# Plot POI pdf vs simulated data
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='PDF', marker_color='black'))
fig.add_trace(go.Histogram(x=var_sim, histnorm='probability density', name='Simulation'))

fig.update_layout(    
    xaxis_title = "x",
    yaxis_title = "P(x)",
    title = 'Generalized Pareto Probability Mass Function',
    yaxis=dict(rangemode='nonnegative')
)
fig.update_xaxes(range=[0, x.max()])
fig.show()

## **Simulate extreme values by considering the simulated PARETO-POISSON parameters** <a class="anchor" id="3"></a>

In [ ]:
# years of Hs extremes to simulate
n_sims = 1000
years_sim = 100 

ds_sim = xr.Dataset()

for c, ts in enumerate(range(n_sims)):
    sys.stdout.write('Simulation {0} from {1}\r'.format(c, len(range(n_sims))))
    sys.stdout.flush()
    
    # empty array to storage annual maximas
    annual_max = []

    # get number of extreme events for each year from POI frequency distribution
    sim_n_events_year = rv_poi.rvs(size=years_sim)
    
    # simulate a large data sample
    size = int(params * 2 * years_sim)
    intensity_events = genpareto.rvs(shape, loc=0, scale=scale, size=size)
    
    # for each year, 
    for y in range(years_sim):

        # simulate intensity from GPD intensity distribution, for all extreme events
        n_events = sim_n_events_year[y]

        # find annual maxima
        if n_events > 0:
            
            # select number of events
            events = intensity_events[0:n_events]
            intensity_events = intensity_events[n_events:]

            annual_max.append(th + events.max())
            
        else: annual_max.append(th)

    # Simulated Hs Extremes
    sim_extremes = pd.DataFrame({var : annual_max})
    sim_extremes.index.name = 'year'
    sim_extremes = sim_extremes.to_xarray().assign_coords(sim=c).expand_dims('sim')

    ds_sim = xr.merge([sim_extremes, ds_sim])


### Plot return period

In [ ]:
# historical rp time and sorted annual maxima
trp_hist = t_rp(imax.index)
trp_hist_val = np.sort(imax[var])

# simulation rp time and sorted annual maxima
trp_sim = t_rp(ds_sim.year.values)
trp_sim_val = np.sort(ds_sim[var].values)

# calculate simulation maxima percentiles
p95 = np.nanpercentile(trp_sim_val, 100-5/2.0, axis=0,)
p50 = np.nanpercentile(trp_sim_val, 50, axis=0,)
p05 = np.nanpercentile(trp_sim_val, 5/2.0, axis=0,)

In [ ]:
# Plot return period

fig = go.Figure()
fig.add_trace(go.Scatter(x=trp_sim, y=p95, mode='lines', name='P95', marker_color='mediumturquoise'))
fig.add_trace(go.Scatter(x=trp_sim, y=p05, mode='lines', name='P05', marker_color='mediumturquoise',  fill='tonexty', fillcolor='rgba(0, 181, 204, 0.10)'))
fig.add_trace(go.Scatter(x=trp_sim, y=p50, mode='lines', name='P50', marker_color='black'))
fig.add_trace(go.Scatter(x=trp_hist, y=trp_hist_val, mode='markers', name='Hist', marker_color='red'))


fig.update_xaxes(type="log")
fig.update_layout(    
    xaxis_title = "Return Period (years)",
    yaxis_title = "TWL (m)",
    title = 'POT',
    width=400*2.5, height=300*2.5
)
fig.show()

In [ ]:
ds_sim

In [ ]:
# plot GEV pdf vs simulated data
fig = go.Figure()
#fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='PDF', marker_color='black'))
fig.add_trace(go.Histogram(x=ds_sim['TWL'], nbinsx = 60, histnorm='probability density', name='Simulation'))

fig.update_layout(    
    xaxis_title = "x",
    yaxis_title = "P(x)",
    title = 'GEV Probability Density Function',
    yaxis=dict(rangemode='nonnegative')
)
fig.show()